In [2]:
import torch
from PIL import Image, ImageDraw
from typing import Tuple
import numpy as np
import time
import cv2


In [1]:
%cd C:/Users/User/document/final_project/drugweb/flask/final_yolo_5

C:\Users\User\document\final_project\drugweb\flask\final_yolo_5


In [3]:
model = torch.hub.load('ultralytics/yolov5', 'custom', path='best.pt', _verbose=False)

Using cache found in C:\Users\User/.cache\torch\hub\ultralytics_yolov5_master
YOLOv5  2023-7-18 Python-3.10.11 torch-2.0.1+cpu CPU

Fusing layers... 
Model summary: 157 layers, 8358625 parameters, 0 gradients, 20.0 GFLOPs
Adding AutoShape... 


In [29]:
import easydict
from pathlib import Path
import os
base_path = Path(os.getcwd())
base_list = base_path/'a.png'
base_list

WindowsPath('C:/Users/User/document/final_project/drugweb/flask/final_yolo_5/a.png')

In [33]:
type(base_list)

pathlib.WindowsPath

In [351]:
%cd C:/Users/User/document/final_project/drugweb/flask/final_yolo_5

C:\Users\User\document\final_project\drugweb\flask\final_yolo_5


In [353]:
import pandas as pd
image_path = base_path/'a.png'
model = torch.hub.load('ultralytics/yolov5', 'custom', path='best.pt')
img = cv2.imread(str(image_path))
results = model(img)
df = results.pandas().xyxy[0]
# data = pd.read_csv('merge_df_4000.csv') # 4000개 데이터
image_df = pd.merge(df,merge_df, how='left', left_on='name', right_on='drug_N')
image_df[['dl_name','dl_material','confidence','품목일련번호', '주성분', '효능','사용 방법', '주의사항', '주의음식', '부작용','di_edi_code']]

Using cache found in C:\Users\User/.cache\torch\hub\ultralytics_yolov5_master
YOLOv5  2023-7-18 Python-3.10.11 torch-2.0.1+cpu CPU

Fusing layers... 
Model summary: 157 layers, 8358625 parameters, 0 gradients, 20.0 GFLOPs
Adding AutoShape... 


KeyError: "['효능', '사용 방법', '주의사항', '주의음식', '부작용'] not in index"

In [182]:
df['name']

0    K-040434
1    K-043508
2    K-042406
3    K-039703
Name: name, dtype: object

### validation 합치기

In [209]:
from pathlib import Path
path = Path('C:/Users/User/document/final_project/preprocessing/v_data')

In [212]:
import zipfile
from pathlib import Path
from glob import glob
path = Path('C:/Users/User/document/final_project/preprocessing/v_data')
def unzip(path):
    path_list = path.rglob('*.zip')
    for paths in (path_list):
        with zipfile.ZipFile(paths, 'r') as zip_ref:
            zip_ref.extractall(str(paths)[:-4])
            # os.remove(zip_ref)

In [213]:
unzip(path)

In [248]:
import json
def to_dataframe(path):
    path_list = path.glob('*/*')
    df_columns =[]

    for paths in path_list:
        json_path = paths.glob('*.json')
        for j in json_path:  # json_path 경로안에 있는 j 하나씩 읽음 
            with open(j, encoding = 'utf-8') as json_file:
                data = json.load(json_file)['images'][0]
                df_columns.append(data)
                break
    df_vaild = pd.DataFrame(df_columns)
    return df_vaild

In [291]:
v_df = to_dataframe(path)
v_df['drug_N']

Function to_dataframe Took 7.7470 seconds


0      K-044727
1      K-044732
2      K-044736
3      K-044746
4      K-044754
         ...   
495    K-044681
496    K-044682
497    K-044699
498    K-044703
499    K-044704
Name: drug_N, Length: 500, dtype: object

In [292]:
v_df.head()

,file_name,width,height,imgfile,drug_N,...,mark_code_front_img,mark_code_back_img,mark_code_front,mark_code_back,change_date
0,K-044727_0_2_0_0_75_000_200.png,976,1280,K-044727_0_2_0_0_75_000_200.png,K-044727,...,,,,,None
1,K-044732_0_2_0_0_70_000_200.png,976,1280,K-044732_0_2_0_0_70_000_200.png,K-044732,...,,,,,20190222
2,K-044736_0_0_0_0_70_000_200.png,976,1280,K-044736_0_0_0_0_70_000_200.png,K-044736,...,,,,,20210609
3,K-044746_0_2_0_0_70_000_200.png,976,1280,K-044746_0_2_0_0_70_000_200.png,K-044746,...,,,,,20190114
4,K-044754_0_0_0_0_60_000_200.png,976,1280,K-044754_0_0_0_0_60_000_200.png,K-044754,...,,,,,20201221


In [303]:
data = pd.read_csv('data/df_merged.csv')
data.head()

,idx,file_name,width,height,imgfile,...,form_code_name,change_date,area,iscrowd,bbox
0,0,K-019461_0_0_0_0_60_000_200.png,976,1280,K-019461_0_0_0_0_60_000_200.png,...,나정,20200930.0,50838,0,"[408, 519, 229, 222]"
1,1,K-019469_0_0_0_0_60_000_200.png,976,1280,K-019469_0_0_0_0_60_000_200.png,...,필름코팅정,20130129.0,72657,0,"[350, 490, 299, 243]"
2,2,K-019553_0_0_0_0_60_000_200.png,976,1280,K-019553_0_0_0_0_60_000_200.png,...,"서방성캡슐제, 펠렛",20201116.0,126169,0,"[249, 406, 449, 281]"
3,3,K-019699_0_0_0_0_60_000_200.png,976,1280,K-019699_0_0_0_0_60_000_200.png,...,필름코팅정,20190225.0,54488,0,"[357, 541, 278, 196]"
4,4,K-019700_0_0_0_0_60_000_200.png,976,1280,K-019700_0_0_0_0_60_000_200.png,...,필름코팅정,20190225.0,94166,0,"[325, 523, 394, 239]"


In [294]:
def col_delete():
    v_df.drop(['drug_S', 'width', 'height', 'mark_code_front', 'mark_code_back', 'line_front','line_back', 'mark_code_front_anal', 'mark_code_back_anal', 'mark_code_front_img', 'mark_code_back_img',
                  'color_class2', 'file_name', 'dl_company_en', 'di_company_mf', 'di_company_mf_en', 'img_regist_ts',
                  'change_date', 'back_color', 'light_color', 'camera_la', 'camera_lo', 'print_back'],
                  axis = 1, inplace = True)
    return v_df
##### fillna #####
# 0으로 채우기
def fill_zero():
    v_df['leng_long'].fillna(0, inplace = True)
    v_df['leng_short'].fillna(0, inplace = True)
    v_df['thick'].fillna(0, inplace = True)
    return v_df
# unknown으로 채우기
def fill_unknown():
    v_df.fillna('unknown', inplace = True)
    return v_df
v_df = col_delete()
v_df = fill_zero()
v_df = fill_unknown()

In [304]:
def col_delete():
    data.drop(['idx','drug_S', 'width', 'height',  'file_name',  'img_regist_ts',
                  'change_date', 'back_color', 'light_color', 'camera_la', 'camera_lo', 'print_back'],
                  axis = 1, inplace = True)
    return data
##### fillna #####
# 0으로 채우기
def fill_zero():
    data['leng_long'].fillna(0, inplace = True)
    data['leng_short'].fillna(0, inplace = True)
    data['thick'].fillna(0, inplace = True)
    return data
# unknown으로 채우기
def fill_unknown():
    data.fillna('unknown', inplace = True)
    return data
data = col_delete()
data = fill_zero()
data = fill_unknown()

In [345]:
result_df = pd.concat([data, v_df])
result_df.columns

Index(['imgfile', 'drug_N', 'drug_dir', 'size', 'dl_idx', 'dl_mapping_code',
       'dl_name', 'dl_name_en', 'img_key', 'dl_material', 'dl_material_en',
       'dl_custom_shape', 'item_seq', 'di_item_permit_date', 'di_class_no',
       'di_etc_otc_code', 'di_edi_code', 'drug_shape', 'thick', 'leng_long',
       'leng_short', 'print_front', 'color_class1', 'form_code_name', 'area',
       'iscrowd', 'bbox', 'id', 'dl_company', 'chart'],
      dtype='object')

In [346]:
result_df =result_df[['drug_N','dl_name','dl_name_en','dl_material','dl_material_en','item_seq','di_class_no','di_edi_code','form_code_name','bbox']]
result_df =result_df.reset_index(drop=True)
result_df

,drug_N,dl_name,dl_name_en,dl_material,dl_material_en,item_seq,di_class_no,di_edi_code,form_code_name,bbox
0,K-019461,미래트리메부틴정 100mg/병,Trimebutine Tab. Mirae,트리메부틴말레산염,Trimebutine,200607006,[02390]기타의 소화기관용약,657500470,나정,"[408, 519, 229, 222]"
1,K-019469,큐레틴정(빌베리건조엑스),Qretin Tab.,빌베리건조엑스,Bilberry Dried Ext.,200607018,[01310]안과용제,"650101140,A28302561",필름코팅정,"[350, 490, 299, 243]"
2,K-019553,콘택골드캡슐 10mg/PTP,Contac Gold Cap.,클로르페니라민말레산염|벨라돈나총알칼로이드|페닐레프린염산염,Phenylephrine Hydrochloride| Belladonna Total ...,200607735,[01410]항히스타민제,642102330,"서방성캡슐제, 펠렛","[249, 406, 449, 281]"
3,K-019699,에스케이코스카플러스정,Cosca Plus Tab. SK,로사르탄칼륨|히드로클로로티아지드,Hydrochlorothiazide| Losartan potassium,200608151,[02140]혈압강하제,"644703770,653802290,A20753971",필름코팅정,"[357, 541, 278, 196]"
4,K-019700,에스케이코스카플러스에프정,Cosca Plus-F Tab. SK,로사르탄칼륨|히드로클로로티아지드,Hydrochlorothiazide| Losartan potassium,200608152,[02140]혈압강하제,"644703780,653802280,A20753981",필름코팅정,"[325, 523, 394, 239]"
...,...,...,...,...,...,...,...,...,...,...
4519,K-044681,바제칼디정,Bazecal D Tab.,바제독시펜아세테이트|콜레칼시페롤농축분말,,201803969,[03990]따로 분류되지 않는 대사성 의약품,648507790,필름코팅정,NaN
4520,K-044682,팜페인파워연질캡슐(나프록센),Pharmpain Power Soft Cap.,나프록센,,201803970,[01140]해열.진통.소염제,055100300,"연질캡슐제, 액상",NaN
4521,K-044699,유니안트디정,Uniant D Tab.,바제독시펜아세테이트|콜레칼시페롤농축분말,,201804004,[03990]따로 분류되지 않는 대사성 의약품,665507030,필름코팅정,NaN
4522,K-044703,아고틴정 25mg,Argotin Tab.,아고멜라틴,Agomelatine,201804018,[01170]정신신경용제,657203390,필름코팅정,NaN


In [347]:
df2 = pd.read_csv('data/e약은요.csv') # 약 내용 데이터 
df2.drop(['제품명','공개일자','수정일자','업체명','이 약은 어떻게 보관해야 합니까?','이 약을 사용하기 전에 반드시 알아야 할 내용은 무엇입니가?'],axis=1,inplace=True)

In [349]:
merge_df = pd.merge(result_df,df2, how='left', left_on='item_seq', right_on='품목일련번호')
merge_df

,drug_N,dl_name,dl_name_en,dl_material,dl_material_en,...,이 약의 효능은 무엇입니까?,이 약은 어떻게 사용합니까?,이 약의 사용상 주의사항은 무엇입니까?,이 약을 사용하는 동안 주의해야 할 약 또는 음식은 무엇입니까?,이 약은 어떤 이상반응이 나타날 수 있습니까?
0,K-019461,미래트리메부틴정 100mg/병,Trimebutine Tab. Mirae,트리메부틴말레산염,Trimebutine,...,"이 약은 식도역류 및 열공헤르니아, 위 십이지장염, 위ㆍ십이지장궤양에 있어서의 소화...","성인은 1회 1~2정(100~200 mg) 1일 3회 식전에 복용합니다.연령, 증상...","갈락토오스 불내성, Lapp 유당분해효소 결핍증 또는 포도당-갈락토오스 흡수장애 등...",NaN,"드물게 변비, 설사, 복명, 구역, 구토, 소화장애, 구갈, 구내마비감, 심계항진,..."
1,K-019469,큐레틴정(빌베리건조엑스),Qretin Tab.,빌베리건조엑스,Bilberry Dried Ext.,...,NaN,NaN,NaN,NaN,NaN
2,K-019553,콘택골드캡슐 10mg/PTP,Contac Gold Cap.,클로르페니라민말레산염|벨라돈나총알칼로이드|페닐레프린염산염,Phenylephrine Hydrochloride| Belladonna Total ...,...,"이 약은 코감기(급성비염), 알레르기성 비염 또는 부비강염에 의한 코막힘, 콧물, ...","만 15세 이상 및 성인은 1회 1캡슐씩 1일 3회, 매 식후에 복용합니다.만 11...","이 약에 과민증 환자, 만 3개월 미만의 영아, MAO 억제제(항우울제, 항정신병제...","MAO 억제제(항우울제, 항정신병제, 감정조절제, 항파킨슨제 등), 다른 비염용 경...","출혈성뇌졸중, 구역·구토, 목마름(지속적이거나 심한), 변비, 식욕부진, 발진·발적..."
3,K-019699,에스케이코스카플러스정,Cosca Plus Tab. SK,로사르탄칼륨|히드로클로로티아지드,Hydrochlorothiazide| Losartan potassium,...,NaN,NaN,NaN,NaN,NaN
4,K-019700,에스케이코스카플러스에프정,Cosca Plus-F Tab. SK,로사르탄칼륨|히드로클로로티아지드,Hydrochlorothiazide| Losartan potassium,...,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
4519,K-044681,바제칼디정,Bazecal D Tab.,바제독시펜아세테이트|콜레칼시페롤농축분말,,...,NaN,NaN,NaN,NaN,NaN
4520,K-044682,팜페인파워연질캡슐(나프록센),Pharmpain Power Soft Cap.,나프록센,,...,"이 약은 류마티양 관절염, 골관절염(퇴행성 관절질환), 강직성 척추염, 건염(힘줄염...","성인은 류마티양 관절염, 골관절염, 강직성 척추염에는 1회 1~2캡슐(250~500...",이 약 또는 아스피린이나 다른 비스테로이드성 소염진통제(COX-2 저해제 포함)에 ...,다른 비스테로이드성 소염진통제 및 나프록센 유도체와 함께 복용하지 마십시오.히단토인...,"드물게 쇽증상(호흡곤란, 혈압강하, 서맥(느린 맥), 식은땀, 부종(부기) 등), ..."
4521,K-044699,유니안트디정,Uniant D Tab.,바제독시펜아세테이트|콜레칼시페롤농축분말,,...,NaN,NaN,NaN,NaN,NaN
4522,K-044703,아고틴정 25mg,Argotin Tab.,아고멜라틴,Agomelatine,...,NaN,NaN,NaN,NaN,NaN


In [356]:
keys = ['이 약의 효능은 무엇입니까?','이 약은 어떻게 사용합니까?','이 약을 사용하기 전에 반드시 알아야 할 내용은 무엇입니가?','이 약의 사용상 주의사항은 무엇입니까?'
        ,'이 약을 사용하는 동안 주의해야 할 약 또는 음식은 무엇입니까?','이 약은 어떤 이상반응이 나타날 수 있습니까?']
values = ['효능','사용 방법','알아야 할 내용','주의사항','주의음식','부작용','성분명(영어)','제조사명','제조사명(영어)','약품 분류','품목기준 코드','전문의약품 여부','EDI코드','제형',
'모양','식별문자(앞)','식별문자(뒤)','색상','정제 분류명','영역']
my_dict = dict(zip(keys, values))
merge_df = merge_df.rename(columns=my_dict)
merge_df

,drug_N,dl_name,dl_name_en,dl_material,dl_material_en,...,효능,사용 방법,주의사항,주의음식,부작용
0,K-019461,미래트리메부틴정 100mg/병,Trimebutine Tab. Mirae,트리메부틴말레산염,Trimebutine,...,"이 약은 식도역류 및 열공헤르니아, 위 십이지장염, 위ㆍ십이지장궤양에 있어서의 소화...","성인은 1회 1~2정(100~200 mg) 1일 3회 식전에 복용합니다.연령, 증상...","갈락토오스 불내성, Lapp 유당분해효소 결핍증 또는 포도당-갈락토오스 흡수장애 등...",NaN,"드물게 변비, 설사, 복명, 구역, 구토, 소화장애, 구갈, 구내마비감, 심계항진,..."
1,K-019469,큐레틴정(빌베리건조엑스),Qretin Tab.,빌베리건조엑스,Bilberry Dried Ext.,...,NaN,NaN,NaN,NaN,NaN
2,K-019553,콘택골드캡슐 10mg/PTP,Contac Gold Cap.,클로르페니라민말레산염|벨라돈나총알칼로이드|페닐레프린염산염,Phenylephrine Hydrochloride| Belladonna Total ...,...,"이 약은 코감기(급성비염), 알레르기성 비염 또는 부비강염에 의한 코막힘, 콧물, ...","만 15세 이상 및 성인은 1회 1캡슐씩 1일 3회, 매 식후에 복용합니다.만 11...","이 약에 과민증 환자, 만 3개월 미만의 영아, MAO 억제제(항우울제, 항정신병제...","MAO 억제제(항우울제, 항정신병제, 감정조절제, 항파킨슨제 등), 다른 비염용 경...","출혈성뇌졸중, 구역·구토, 목마름(지속적이거나 심한), 변비, 식욕부진, 발진·발적..."
3,K-019699,에스케이코스카플러스정,Cosca Plus Tab. SK,로사르탄칼륨|히드로클로로티아지드,Hydrochlorothiazide| Losartan potassium,...,NaN,NaN,NaN,NaN,NaN
4,K-019700,에스케이코스카플러스에프정,Cosca Plus-F Tab. SK,로사르탄칼륨|히드로클로로티아지드,Hydrochlorothiazide| Losartan potassium,...,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
4519,K-044681,바제칼디정,Bazecal D Tab.,바제독시펜아세테이트|콜레칼시페롤농축분말,,...,NaN,NaN,NaN,NaN,NaN
4520,K-044682,팜페인파워연질캡슐(나프록센),Pharmpain Power Soft Cap.,나프록센,,...,"이 약은 류마티양 관절염, 골관절염(퇴행성 관절질환), 강직성 척추염, 건염(힘줄염...","성인은 류마티양 관절염, 골관절염, 강직성 척추염에는 1회 1~2캡슐(250~500...",이 약 또는 아스피린이나 다른 비스테로이드성 소염진통제(COX-2 저해제 포함)에 ...,다른 비스테로이드성 소염진통제 및 나프록센 유도체와 함께 복용하지 마십시오.히단토인...,"드물게 쇽증상(호흡곤란, 혈압강하, 서맥(느린 맥), 식은땀, 부종(부기) 등), ..."
4521,K-044699,유니안트디정,Uniant D Tab.,바제독시펜아세테이트|콜레칼시페롤농축분말,,...,NaN,NaN,NaN,NaN,NaN
4522,K-044703,아고틴정 25mg,Argotin Tab.,아고멜라틴,Agomelatine,...,NaN,NaN,NaN,NaN,NaN


In [155]:
df_warming = pd.read_csv('warming_df.csv') # 주의 데이터
df_warming['code조합'] = df_warming['제품코드A'].astype(str) + ',' + df_warming['제품코드B'].astype(str)
df_warming['code조합']

0          648101510,646002281
1          648101510,647804481
2          648101510,642305011
3           648101510,59000681
4          648101510,653400910
                  ...         
1564057    653403230,641105840
1564058    654004590,641105840
1564059    655403650,641105840
1564060    655604700,641105840
1564061    656003820,641105840
Name: code조합, Length: 1564062, dtype: object

In [165]:
test_lst = ['648101510','646002281']
test_lst.isin(df_warming['code조합'].iloc[1].values)

AttributeError: 'list' object has no attribute 'isin'

In [185]:
def check_alarm():
    for value in test_lst:
        if any(value in s for s in df_warming['code조합']):
            return 
    return False


ALARM: List contains a value from the Series!


In [179]:
# indexes_with_target_list = [index for index, value in df_warming['code조합'].items() if test_lst in value]
# indexes_with_target_list
indexes_with_target_list = [index for index, value in df_warming['code조합'].items() if all(item in value for item in test_lst)]
df_warming.loc[indexes_with_target_list]['상세정보']

0          중증의 위장관계 이상반응
465674     중증의 위장관계 이상반응
1016081    중증의 위장관계 이상반응
Name: 상세정보, dtype: object

In [158]:
index_1 = image_df[image_df['di_edi_code'].isin(df_warming['code조합'].values)]['di_edi_code'].index
index_1

Index([], dtype='int64')

In [157]:
if df['di_edi_code'].isin(df_warming['code조합'].values) == True:
    index_1 = image_df[image_df['di_edi_code'].isin(df_warming['code조합'].values)]['di_edi_code'].index
    
else:
    print()

0       False
1       False
2       False
3       False
4       False
        ...  
4019    False
4020    False
4021    False
4022    False
4023    False
Name: di_edi_code, Length: 4024, dtype: bool

In [90]:
# lst =[]
# for i in data['file_name']:
#     i = i[0:8]
#     lst.append(i)
# data['name'] = lst

In [91]:
image_df = pd.merge(df,data, how='left', left_on='name', right_on='name')

In [94]:
image_df.columns

Index(['xmin', 'ymin', 'xmax', 'ymax', 'confidence', 'class', 'name', 'idx',
       'file_name', 'width', 'height', 'imgfile', 'drug_N', 'drug_S',
       'back_color', 'drug_dir', 'light_color', 'camera_la', 'camera_lo',
       'size', 'dl_idx', 'dl_mapping_code', 'dl_name', 'dl_name_en', 'img_key',
       'dl_material', 'dl_material_en', 'dl_custom_shape', 'item_seq',
       'di_item_permit_date', 'di_class_no', 'di_etc_otc_code', 'di_edi_code',
       'drug_shape', 'thick', 'leng_long', 'leng_short', 'print_front',
       'print_back', 'color_class1', 'img_regist_ts', 'form_code_name',
       'change_date', 'area', 'iscrowd', 'bbox', '품목일련번호', '주성분', '효능',
       '사용 방법', '주의사항', '주의음식', '부작용'],
      dtype='object')

In [121]:
df['name']

0    K-040434
1    K-043508
2    K-042406
3    K-039703
Name: name, dtype: object

In [128]:
data['name'].value_counts()

name
K-029066    2
K-019461    1
K-027055    1
K-027007    1
K-027008    1
           ..
K-010294    1
K-010295    1
K-010305    1
K-010316    1
K-019299    1
Name: count, Length: 4023, dtype: int64